# INTRODUCTION TO OPTIMIZATION 

## Picking the right language
Let's see a problem that takes time, adding up the first billion numbers. 

In Python you can do it by using a loop. 

In [1]:
import time
N = 1000_000_000

Let's try to add them up "manually". 

In [ ]:
start = time.time()

total_loop = 0
for i in range(N):
    total_loop += i

print(f"Loop total: {total_loop}")
print(f"Loop time: {time.time() - start} s")

Loop total: 499999999500000000
Loop time: 46.79621720314026 s


Note that it took some time even on a strong machine. Let's try the same with Python's native `sum()` function.

In [9]:
start = time.time()
total_sum = sum(range(N))
print(f"Sum total: {total_sum}")
print(f"Sum time: {time.time() - start} s")

Sum total: 499999999500000000
Sum time: 7.801518201828003 s


This was much faster. When python runs this function, there is a low level language running in the background, similar to this `C++` script:

```cpp
#include <iostream>
#include <chrono>

int main() {
auto start = std::chrono::high_resolution_clock::now();
long long total = 0;
for (int i = 0; i < 1000000000; ++i) total += i;
std::cout << total << std::endl;
auto end = std::chrono::high_resolution_clock::now();
std::chrono::duration<double> diff = end - start;
std::cout << "Time: " << diff.count() << " s\n";
return 0;
}
```

### Interpreted vs Compiled languages

A **compiled language** (like C++) translates the entire code into machine instructions **before** it runs. So when you execute it, it's already in the fastest form your computer understands.

An **interpreted language** (like Python) reads and executes code **line by line**, while the program is running. This makes it more flexible and easy to write, but slower because it's doing more work at runtime.

That’s why Python is often used for prototyping. It may be too slow for really big tasks, but it's easy to write and debug. Once the code is running as intended, you can "translate" it to a low level language. 

### JIT - Just In Time Compilation
There are languages that were developed specifically for scientific computing, such as Matlab or Julia. The latter is especially known for being "easy as Python, fast as C". 

Julia looks like an interpreted language (just like Python), but in reality it uses **just in time compliation (JIT)**. It doesn't compile the entire code before it runs (like C), but compiles parts of it as needed, on the go. 

While Juila or Matlab were designed to be able to do this there are also Python packages (such as `numba`) that enable JIT.

```julia
@time total = sum(0:999_999_999)
println(total)
```
(Note that Julia uses 1-indexing instead of 0-indexing, similar to R.)

You may notice that the runtime was extremely low. But how can it be even faster than a language close to machine code?

In scientific computing languages like Julia, many mathematical patterns are recognized and mapped to hardcoded optimizations. In this case, Julia detects that ranges like `(n:m)` are a special case for `sum()` and uses a fast, pre-optimized formula instead of looping (here a `UnitRange` method).

Conceptually, this is similar to the trick the little Gauss allegedly used to sum numbers quickly:

$\sum_{i=1}^{n} i = \frac{n(n+1)}{2}$

Take home message: using your brain can speed up your code. 🙃

In [8]:
start = time.time()

n = N - 1
total_sum = n * (1 + n) // 2
print(f"Sum total: {total_sum}")
print(f"Sum time: {time.time() - start} s")

Sum total: 499999999500000000
Sum time: 0.00015211105346679688 s


See?

### Self check questions
When is it worth to use a compiled languge?<br>
Why is Python widely used in scientific tasks despite its slow performance?<br>
What makes Julia "as easy as Python and as fast as C"?<br>
What happens when you sum random numbers instead of a range?

## Introduction to parallel computing

Add up 100 thousand random numbers.

In [ ]:
from time import time
from random import random, seed
seed(42)

start = time()
print(sum([random() for _ in range(100_000_000)]))
print(time() - start)

49998298.34697442
5.654012203216553


In [ ]:
import psutil
print("Logical CPUs (threads):", psutil.cpu_count(logical=True))
print("Physical cores:", psutil.cpu_count(logical=False))

Logical CPUs (threads): 8
Physical cores: 8


Unfortunately, multiprocessing and Jupyter notebook don't go well hand-in-hand, so consult the README.md for this part.

## Generate random FASTA

In [10]:
import random

def generate_fasta(file_path, num_sequences=1000, seq_length=1000):
    bases = ['A', 'C', 'G', 'T']
    with open(file_path, 'w') as f:
        for i in range(num_sequences):
            seq_id = f">seq{i+1}\n"
            sequence = ''.join(random.choices(bases, k=seq_length)) + '\n'
            f.write(seq_id)
            f.write(sequence)

output_path = '../data/sequences.fasta'
generate_fasta(output_path, num_sequences=10, seq_length=1000)
print(f"FASTA file generated at {output_path}")

FASTA file generated at ../data/sequences.fasta


In [9]:
import torch
torch.backends.mps.is_available()

/Users/rblc/ELTE/optimization_class/.venv/lib/python3.11/site-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


True